# PDF Ingestion Pipeline

## 1. Instruction for running on local machine
1. Create a virtual environment (Recommended)
2. Make sure you have install all dependencies listed in requirements.txt
3. In addition, for reading PDFs and OCR, we'll need Poppler, Tesseract, and Tesseract-lang. If using Homebrew, you can run the following commands

```bash
brew install poppler
brew install tesseract-lang
```

4. Make sure you have your PDF files placed in folder `data`

## 2. Instruction for Google Colab
TODO

In [2]:
import os
import pprint
from dotenv import load_dotenv
from unstructured.partition.pdf import partition_pdf
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_milvus.vectorstores import Milvus
load_dotenv()

In [ ]:
# GLOBALS
file_path = "./data/test.pdf"

In [3]:
partitioned = UnstructuredPDFLoader(
    file_path,
    mode="elements",
    languages=["vie", "equ"]
)

In [4]:
partitioned

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    length_function=len,
)

In [6]:
docs = partitioned.load_and_split(text_splitter)

In [8]:
len(docs)

99

In [13]:
docs[90]

Document(metadata={'source': './data/test.pdf', 'coordinates': {'points': ((83.24999653124999, 692.4862960736531), (83.24999653124999, 704.6137956433406), (139.32936923210934, 704.6137956433406), (139.32936923210934, 692.4862960736531)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 792}, 'file_directory': './data', 'filename': 'test.pdf', 'languages': ['vie', 'equ'], 'last_modified': '2024-07-22T22:34:59', 'page_number': 1, 'parent_id': 'af8176af373c3242cf3dbc7cdb74a3d7', 'filetype': 'application/pdf', 'category': 'UncategorizedText'}, page_content='2x − 3 > 1')

In [11]:
pprint.pprint([page.page_content for page in docs])

['7/13/24, 1:14 PM',
 'question_set (5).html',
 'SỞ GD&ĐT HÀ GIANG TRƯỜNG PTDTNT THCS VÀ THPT HUYỆN ĐỒNG VĂN',
 'Đề chính thức (Đề gồm 01 trang)',
 'ĐỀ KIỂM TRA GIỮA KÌ HỌC KÌ II Năm học: 2023 - 2024 Môn: Toán Lớp: 9 Thời '
 'gian: 90 phút (Không kể thời gian giao đề)',
 'I. Trắc nghiệm: (4,0 điểm). Chọn câu trả lời đúng nhất trong các câu sau:',
 'Câu 1 (4 điểm) Cho bất đẳng thức',
 'a ≤ b',
 '. Phát biểu nào sau đây đúng?',
 'A.',
 'a',
 'luôn nhỏ hơn',
 'b',
 'B.',
 'a',
 'b có thể bằng',
 'C.',
 'a',
 'luôn lớn hơn',
 'b',
 'a',
 'D. hoặc bằng',
 'có thể lớn hơn b',
 'Câu 2 (8 điểm) Nếu',
 'a ≤ b',
 'và',
 'b ≤ c',
 ', phát biểu nào sau đây là đúng?',
 'A.',
 'a ≥ c',
 'B.',
 'a = c',
 'C.',
 'a ≤ c',
 'c D. quan đến nhau',
 'a',
 'và không liên',
 'Câu 3 (4 điểm) Phương trình nào sau đây có nghiệm',
 'x = 2 ?',
 'A.',
 'x + 2 = 0',
 'B.',
 '2x − 4 = 0',
 'C.',
 'x2 − 4 = 0',
 'D.',
 '3x + 1 = 7',
 'Câu 4 (8 điểm) Cho phương trình',
 '2x − 5 = 3',
 '. Giá trị nào của',
 'x',
 'thỏa

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name='keepitreal/vietnamese-sbert')

/Users/mynguyen/Repos/3T/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [14]:
embedding_model.embed_query(docs[90].page_content)

[-0.28402501344680786,
 -0.5487903952598572,
 -0.0010314807295799255,
 -0.2683267891407013,
 -0.23845525085926056,
 0.03744678944349289,
 0.5013006925582886,
 0.036411263048648834,
 -0.10451315343379974,
 -0.04259064421057701,
 0.20969325304031372,
 0.08021889626979828,
 0.2582714259624481,
 -0.010935315862298012,
 0.6217290759086609,
 -0.2559450566768646,
 -0.25258883833885193,
 -0.23963870108127594,
 0.3638308644294739,
 0.002480447292327881,
 -0.26570165157318115,
 0.27283912897109985,
 -0.19392363727092743,
 -0.11761990934610367,
 0.9814473986625671,
 -0.5258400440216064,
 -0.28609803318977356,
 0.13027453422546387,
 -0.14989250898361206,
 0.4806583821773529,
 0.4291461706161499,
 0.20437800884246826,
 -0.06361537426710129,
 0.1197303980588913,
 -0.3130582869052887,
 -0.31854668259620667,
 0.09592252224683762,
 -0.2514815330505371,
 -0.32245856523513794,
 0.1683950126171112,
 0.3556184768676758,
 -0.16285967826843262,
 0.5211061835289001,
 0.31095343828201294,
 0.5436096787452698,


In [ ]:
URI = os.environ.get("PUBLIC_ENDPOINT")
vector_db = Milvus.from_documents(
    docs,
    embedding_model,
    collection_name="python_oh",
    connection_args={"uri": URI,
                     "user": os.environ.get("USER"),
                     "password": os.environ.get("PASSWORD")},
)

In [ ]:
test_query = "Mục lục là gì thế?"
docs = vector_db.similarity_search(test_query)